In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from profit.sur.backend.gp_functions import invert, nll, predict_f, \
    get_marginal_variance_BBQ, wld_get_marginal_variance
from profit.sur.backend.kernels import kern_sqexp
from profit.util.halton import halton


In [ ]:
def f(x): return x*np.cos(10*x)

# Custom function to build GP matrix
def build_K(xa, xb, hyp, K):
    for i in np.arange(len(xa)):
        for j in np.arange(len(xb)):
            K[i, j] = kern_sqexp(xa[i], xb[j], hyp[0])

noise_train = 0.0

ntrain = 15
xtrain = halton(1, ntrain)
ftrain = f(xtrain)
np.random.seed(0)
ytrain = ftrain + noise_train*np.random.randn(ntrain, 1)

In [ ]:
# GP regression with fixed kernel hyperparameters
hyp = [0.1, 1e-3]  # l and sig_noise**2

K = np.empty((ntrain, ntrain))   # train-train
build_K(xtrain, xtrain, hyp, K)  # writes inside K
Ky = K + hyp[-1]*np.eye(ntrain)
Kyinv = invert(Ky, 4, 1e-6)       # using gp_functions.invert

ntest = 20
xtest = np.linspace(0, 1, ntest)
ftest = f(xtest)

Ks = np.empty((ntrain, ntest))  # train-test
Kss = np.empty((ntest, ntest))  # test-test
build_K(xtrain, xtest, hyp, Ks)
build_K(xtest, xtest, hyp, Kss)

fmean = Ks.T.dot(Kyinv.dot(ytrain)) # predictive mean


In [ ]:
plt.figure()
plt.plot(xtrain, ytrain, 'x')
plt.plot(xtest, ftest, '-')
plt.plot(xtest, fmean, '--')
plt.legend(('training', 'reference', 'prediction'))

In [ ]:
Ef, varf = predict_f(hyp, xtrain.reshape(-1, 1),
                      ytrain.reshape(-1, 1), xtest.reshape(-1, 1), neig=8)# posterior 
# Estimation and variance
varf = np.diag(varf)

# we keep only the diag because the variance is on it, the other terms are covariance


plt.plot(xtrain, ytrain, 'kx')
plt.plot(xtest, ftest, 'm-')
plt.plot(xtest, fmean, 'r--')
axes = plt.gca()
axes.set_ylim([-1.5, 1])
plt.title('Gaussian Process with '+ str(ntrain) + ' observation(s)')
plt.legend(('training', 'reference', 'prediction'))



plt.fill_between(xtest, # x
                 (fmean.flatten() + 2 * np.sqrt(varf)), # y1
                 (fmean.flatten() - 2 * np.sqrt(varf))) # y2

In [ ]:
# Negative log likelihood over length scale
ls = np.linspace(1e-3, 3, 50)
nlls = np.array(
    [nll([l, 1e-3], xtrain, ytrain) for l in ls]
    ).flatten()

In [ ]:
plt.figure()
plt.plot(ls, nlls)
plt.xlabel('l')
plt.ylabel('- log p(y|l)')
plt.title('Negative log-likelihood')

In [ ]:
from scipy.optimize import minimize

# Prior to cut out range
def cutoff(x, xmin, xmax, slope=1e3):
    if x < xmin:
        return slope*(x - xmin)**2
    if x > xmax:
        return slope*(x - xmax)**2
    
    return 0.0

def nlprior(log10hyp):
    return cutoff(log10hyp[0], -2, 1) + cutoff(log10hyp[-1], -8, 0)

x = np.linspace(-10, 1, 100)
plt.figure()
plt.plot(x, [cutoff(xi, -6, 0) for xi in x])
plt.show()

def nlp_transform(log10hyp):
    hyp = 10**log10hyp
    return nll(hyp, xtrain, ytrain) + nlprior(log10hyp)

res = minimize(nlp_transform, np.array([-1, -6]), method='BFGS')

In [ ]:
print(res)
#hyp = 10**res.x
print('[l,sig2] = ', 10**res.x)
print('[log_l, log_s2] = ', res.x)
log_l = res.x[0]
log_s2= res.x[1]
log_hyp = [log_l, log_s2]

new_hyp = [10**res.x[0], 10**res.x[1]]
hess_inv = res.hess_inv
print("\nhess_inv = ", hess_inv)
print("\nhess = ", invert(hess_inv))

In [ ]:
nl = 50
ns2 = 40

log10l = np.linspace(res.x[0]-1, res.x[0]+1, nl)
log10s2 = np.linspace(res.x[1]-1, res.x[1]+1, ns2)
[Ll, Ls2] = np.meshgrid(log10l, log10s2)

nlls = np.array(
    [nlp_transform(np.array([ll, ls2])) for ls2 in log10s2 for ll in log10l]
    ).reshape([ns2, nl])
    
# Do some cut for visualization
maxval = 0.0
nlls[nlls>maxval] = maxval

plt.figure()
plt.title('NLL')
plt.contour(Ll, Ls2, nlls, levels=50)
plt.plot(res.x[0], res.x[1], 'rx')
plt.xlabel('log10 l^2')
plt.ylabel('log10 sig_n^2')
plt.colorbar()
plt.legend(['optimum'])
plt.show()

In [ ]:
nlls

In [ ]:
from profit.sur.backend.gp_functions import k

def dk_logdl(xa, xb, l): # derivative of the kernel w.r.t log lengthscale
    dk_dl = ((xa - xb)**2.0 * np.exp(-(xa-xb)**2.0/(2 * l**2))) / l**3 
    dk_logdl = dk_dl * np.log(10) * 10**log_l # from log lengthscale to lengthscale
    return dk_logdl
    

def dkdl(xa, xb, l): # derivative of the kernel w.r.t lengthscale
    dk_dl = ((xa - xb)**2.0 * np.exp(-(xa-xb)**2.0/(2 * l**2))) / l**3 
    return dk_dl

In [ ]:

log_K = np.empty((ntrain, ntrain))
for i in np.arange(len(xtrain)):
    
    for j in np.arange(len(xtrain)):
        log_K[i, j] = k(xtrain[i], xtrain[j], log_hyp[0])


log_K_star = np.empty((ntest, ntrain))
for i in np.arange(len(xtest)):
    for j in np.arange(len(xtrain)):
        log_K_star[i, j] = k(xtest[i], xtrain[j], log_hyp[0])


log_K_prime = np.empty((ntrain, ntrain))
for i in np.arange(len(xtrain)):
    for j in np.arange(len(xtrain)):
        log_K_prime[i, j] = dk_logdl(xtrain[i], xtrain[j], log_hyp[0])


log_K_star_prime = np.empty((ntest, ntrain))
for i in np.arange(len(xtest)):
    for j in np.arange(len(xtrain)):
        log_K_star_prime[i, j] = dk_logdl(xtest[i], xtrain[j], log_hyp[0])




# K.shape = (20, 20)
# K_prime.shape = (20, 20)

# K_star.shape = (10, 20)
# K_star_prime.shape = (10, 20)

In [ ]:


log_Ky = log_K + log_hyp[-1]*np.eye(ntrain)
log_Kyinv = invert(log_Ky, 4, 1e-6)

alpha = np.dot(Kyinv, ytrain) # RW p17 paragraph 4

log_dalpha_dl = -Kyinv.dot(log_K_prime)\
    .dot(Kyinv)\
    .dot(ytrain)

log_dalpha_ds = -Kyinv.dot(np.log(10) * 10**log_s2 * np.eye(ntrain)).dot(Kyinv).dot(ytrain) # - Kyinv x ln(10) x 10^log_sigma x I x Kyinv x ytrain 

log_dm = np.empty((ntest,len(hyp), 1))

for nb_hyp in range(len(hyp)):
    if nb_hyp == 0 :
        log_dm[:,nb_hyp,:] = np.dot(log_K_star_prime, alpha) -\
                         np.dot(log_K_star, log_dalpha_dl) 
    else : 
        log_dm[:,nb_hyp,:] = np.dot(log_K_star, log_dalpha_ds)

print("\n\n\ndm :",log_dm.shape, "\n\n")
#print(dm)

    




In [ ]:
log_sigma = invert(hess_inv) # define the sigma matrix as the inverse of hess_inv
V = varf # set V as the result of the predict_f diagonal  

#print("\nsigma shape : ", log_sigma.shape)
#print(log_sigma)
#print("dm.shape : ", log_dm.shape)


log_dm_transpose = np.empty((ntest, 1, len(log_hyp)))
log_dmT_dot_sigma = np.empty((ntest, 1, len(log_hyp)))
log_dmT_dot_sigma_dot_dm = np.empty((ntest, 1))



for i in range(ntest):
    log_dm_transpose[i] = log_dm[i].T
    #print("\n\ndm.t",i,' : ', log_dm_transpose[i])
    log_dmT_dot_sigma[i] = log_dm_transpose[i].dot(log_sigma)
    #print("sigma = ", log_sigma)
    #print("dmT_dot_sigma",i," : ", log_dmT_dot_sigma[i])
    log_dmT_dot_sigma_dot_dm[i] = log_dmT_dot_sigma[i].dot(log_dm[i])
    #print("dmT_dot_sigma_dot_dm",i," : ", dmT_dot_sigma_dot_dm[i])
    
# print("dm_transpose :", dm_transpose.shape)
# print("\ndmT_dot_sigma ", dmT_dot_sigma.shape)
# print("dmT_dot_sigma_dot_dm ", dmT_dot_sigma_dot_dm.shape)
# print("V ", V.shape)


 
log_V_tild = V.reshape((ntest,1)) + log_dmT_dot_sigma_dot_dm # Osborne et al. (2012) Active learning eq.19 


print("V_tild.shape ", log_V_tild.shape)
print("\n\n\tMarginal variance\n\n", log_V_tild )




In [ ]:
import time

tac = time.time()
marginal_variance = get_marginal_variance_BBQ(hess_inv, new_hyp, ntrain, ntest, xtrain, xtest, 
                                          Kyinv, ytrain, varf, True)
tuc = time.time()

log_time = tuc - tac 
print(log_time, " second\n-> ", log_time * 1000, " ms")


In [ ]:

plt.plot(xtrain, ytrain, 'kx')
plt.plot(xtest, ftest, 'm-')
plt.plot(xtest, fmean, 'r--')
axes = plt.gca()
axes.set_ylim([-2, 2])
plt.title('Gaussian Process with '+ str(ntrain) + ' observation(s)')
plt.legend(('training', 'reference', 'prediction'))



plt.fill_between(xtest, # x
                 (fmean.flatten() + 2 * np.sqrt(marginal_variance.flatten())), # y1
                 (fmean.flatten() - 2 * np.sqrt(marginal_variance.flatten()))) # y2


In [ ]:
def wld_get_marginal_variance(wld_hess_inv, wld_hyp, ntrain, ntest, xtrain, xtest,
                              Kyinv, ytrain, varf, plot_result = False):



    ######################### Build needed Kernel Matrix #########################
    wld_K = np.empty((ntrain, ntrain))
    for i in np.arange(len(xtrain)):
        for j in np.arange(len(xtrain)):
            wld_K[i, j] = k(xtrain[i], xtrain[j], wld_hyp[0])


    wld_K_star = np.empty((ntest, ntrain))
    for i in np.arange(len(xtest)):
        for j in np.arange(len(xtrain)):
            wld_K_star[i, j] = k(xtest[i], xtrain[j], wld_hyp[0])


    wld_K_prime = np.empty((ntrain, ntrain))
    for i in np.arange(len(xtrain)):
        for j in np.arange(len(xtrain)):
            wld_K_prime[i, j] = dkdl(xtrain[i], xtrain[j], wld_hyp[0])


    wld_K_star_prime = np.empty((ntest, ntrain))
    for i in np.arange(len(xtest)):
        for j in np.arange(len(xtrain)):
            wld_K_star_prime[i, j] = dkdl(xtest[i], xtrain[j], wld_hyp[0])
    ############################################################################


    wld_alpha = np.dot(Kyinv, ytrain) # RW p17 paragraph 4

    wld_dalpha_dl = -Kyinv.dot(wld_K_prime)\
        .dot(Kyinv)\
        .dot(ytrain)

    wld_dalpha_ds = -Kyinv.dot(np.eye(ntrain)).dot(Kyinv).dot(ytrain) # - Kyinv x I x Kyinv x ytrain

    wld_dm = np.empty((ntest,len(wld_hyp), 1))


    for nb_hyp in range(len(wld_hyp)):
        if nb_hyp == 0 :
            wld_dm[:,nb_hyp,:] = np.dot(wld_K_star_prime, wld_alpha) -\
                             np.dot(wld_K_star, wld_dalpha_dl)
        else :
            wld_dm[:,nb_hyp,:] = np.dot(wld_K_star, wld_dalpha_ds)

    V = varf # set V as the result of the predict_f diagonal
    wld_sigma = invert(wld_hess_inv)
    print("sigma ", wld_sigma)

    wld_dm_transpose = np.empty((ntest, 1, len(wld_hyp)))
    wld_dmT_dot_sigma = np.empty((ntest, 1, len(wld_hyp)))
    wld_dmT_dot_sigma_dot_dm = np.empty((ntest, 1))

    for i in range(ntest):
        wld_dm_transpose[i] = wld_dm[i].T
        wld_dmT_dot_sigma[i] = wld_dm_transpose[i].dot(wld_sigma)
        wld_dmT_dot_sigma_dot_dm[i] = wld_dmT_dot_sigma[i].dot(wld_dm[i])

    wld_V_tild = V.reshape((ntest,1)) + wld_dmT_dot_sigma_dot_dm # Osborne et al. (2012) Active learning eq.19

    if plot_result == True :
        print("The marginal Variance has a shape of ", wld_V_tild.shape)
        print("\n\n\tMarginal variance\n\n", wld_V_tild )

    return wld_V_tild



result = minimize(nll, hyp, args=(xtrain, ytrain), method='L-BFGS-B') 
# Got Identity matrix as hessian with L-BFGS-B
print("\n\n", result)
wld_hyp = result.x
wld_hess_inv = result.hess_inv.todense()

tic = time.time()

wld_marginal_variance = wld_get_marginal_variance(wld_hess_inv, wld_hyp, ntrain, 
                                 ntest, xtrain, xtest, Kyinv, ytrain, varf, True)

tac = time.time()

wld_time = tac - tic 
print(wld_time, " second\n-> ", wld_time * 1000, " ms")

In [ ]:
diff_time = np.abs(wld_time - log_time)
diff = []
res = 0
for i in range(len(marginal_variance)):
    res += np.abs(marginal_variance[i] - wld_marginal_variance[i])/wld_marginal_variance[i]

res *= 100
res /= len(marginal_variance)

print("\tThere is a value difference of ",round(res.item(), 1), " % ")
print("\tWld method time : ", round(wld_time*1000, 2), " ms")
print("\tLog method time : ", round(log_time*1000, 2), " ms")
    
    
    

In [ ]:
plt.plot(xtrain, ytrain, 'kx')
plt.plot(xtest, ftest, 'm-')
plt.plot(xtest, fmean, 'r--')
axes = plt.gca()
axes.set_ylim([-2, 2])
plt.title('Gaussian Process with '+ str(ntrain) + ' observation(s)')
plt.legend(('training', 'reference', 'prediction'))



plt.fill_between(xtest, # x
                 (fmean.flatten() + 2 * np.sqrt(wld_marginal_variance.flatten())), # y1
                 (fmean.flatten() - 2 * np.sqrt(wld_marginal_variance.flatten()))) # y2


In [ ]:

# Step 1 : invert the res.hess_inv to get H_tilde
H_tilde = invert(hess_inv)
# Step 2 Get H
H = np.zeros((len(H_tilde), len(H_tilde)))
for i in np.arange(len(H_tilde)):
    for j in np.arange(len(H_tilde)):
        H[i,j] = (1/np.log(10)**2) * H_tilde[i,j]/(new_hyp[i]*new_hyp[j])
# Step 3 get Sigma
H_inv = invert(H)
sigma_m = H_inv


######################### Build needed Kernel Matrix #########################

# Kernel K(train, train) of shape (ntrain, ntrain)
K = np.empty((ntrain, ntrain))
for i in np.arange(len(xtrain)):
    for j in np.arange(len(xtrain)):
        K[i, j] = k(xtrain[i], xtrain[j], new_hyp[0])

# Kernel K_star(test, train) of shape (ntest, ntrain)
# note that K_star(test, train) = K_star.T(train, test)
K_star = np.empty((ntest, ntrain))
for i in np.arange(len(xtest)):
    for j in np.arange(len(xtrain)):
        K_star[i, j] = k(xtest[i], xtrain[j], new_hyp[0])


# Derivative of kernel K
K_prime = np.empty((ntrain, ntrain))
for i in np.arange(len(xtrain)):
    for j in np.arange(len(xtrain)):
        K_prime[i, j] = dkdl(xtrain[i], xtrain[j], new_hyp[0])

# Derivative of kernel K_star
K_star_prime = np.empty((ntest, ntrain))
for i in np.arange(len(xtest)):
    for j in np.arange(len(xtrain)):
        K_star_prime[i, j] = dkdl(xtest[i], xtrain[j], new_hyp[0])

K_2star_prime = np.empty((ntest, ntest))
for i in np.arange(len(xtest)):
    for j in np.arange(len(xtest)):
        K_2star_prime[i, j] = dkdl(xtest[i], xtest[j], new_hyp[0])
############################################################################



dKyinv_dl = -Kyinv.dot(K_prime).dot(Kyinv) # Compute the Kyinv's derivative w.r.t. lengthscale

dKyinv_ds = -Kyinv.dot(np.eye(ntrain)).dot(Kyinv) # Compute the Kyinv's derivative w.r.t. sigma noise
                                                              # square
dKyinv = np.concatenate([dKyinv_dl[None,:,:], dKyinv_ds[None,:,:]], axis=0)
print("dKyinv shape : ", dKyinv.shape)


term2_up = K_star.dot(dKyinv_dl).dot(K_star.T)
term2_down = K_star.dot(dKyinv_ds).dot(K_star.T)
term2 = np.concatenate([term2_up [None,:,:], term2_down[None,:,:]], axis=0)


term1 = K_star_prime.dot(Kyinv).dot(K_star.T) 

term3 = K_star.dot(Kyinv).dot(K_star_prime.T)

print("term1 ", term1.shape)
print("term2 ", term2.shape)
print("term3 ", term3.shape)

tot = term1 + term2 + term3
print("tot ", tot.shape)


In [ ]:
nl = 50
ns2 = 40


log10s2 = np.linspace(res.x[1]-1, res.x[1]+4, ns2)
[Ll, Ls2] = np.meshgrid(log10l, log10s2)

nlls = np.array(
    [nll([10**log_l, 10**ls2], xtrain, ytrain, 0) for ls2 in log10s2]
    ).reshape([ns2, 1])
    
# Do some cut for visualization
maxval = 0.0
nlls[nlls>maxval] = maxval

plt.figure()
plt.title('NLL with fixed lengthscale')
plt.plot(res.x[0], res.x[1], 'rx')
plt.plot(log10s2, nlls)
plt.xlabel('log10 s^^')
plt.ylabel('nll')
plt.legend(['optimum'])
plt.show()

In [ ]:
nl = 50
ns2 = 40

log10l = np.linspace(res.x[0]-1, res.x[0]+1, nl)

[Ll, Ls2] = np.meshgrid(log10l, log10s2)

nlls = np.array(
    [nll([10**ll, log_s2], xtrain, ytrain, 0) for ll in log10l]
    ).reshape([nl, 1])
    
# Do some cut for visualization
maxval = 0.0
nlls[nlls>maxval] = maxval

plt.figure()
plt.title('NLL')

plt.plot(res.x[0], res.x[1], 'rx')
plt.plot(log10l, nlls)
plt.xlabel('log10 l^2')
plt.ylabel('nll')
plt.legend(['optimum'])
plt.show()


In [ ]:

def nll_fixed_l(new_s2, new_l): 
    return nll([new_l, new_s2], xtrain, ytrain, 0)

lengthscale = np.linspace(0.1,10,100)
sigma_noise = []

for i in lengthscale:
    opti = minimize(nll_fixed_l,1e-8, args=(i), method='BFGS')
    sigma_noise.append(opti.x.item())

plt.figure()
plt.title('Optimized s2 with l fixed')
plt.semilogx(lengthscale, sigma_noise)
plt.xlabel('lengthscale')
plt.ylabel('optimal sigmanoise')
plt.savefig('fixed_l')
plt.show()

In [ ]:
nll(new_hyp,xtrain, ytrain, 0)